In [1]:
# Following variables can be set here or via papermill
# experiment = "experiment-baseline-with-latency-3"

# Imports

In [50]:
from google.oauth2 import service_account
from google.cloud import monitoring_v3
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud.bigquery.job import ExtractJobConfig
import time
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import itertools
import os
from pathlib import Path
import datetime
import shutil
import subprocess
from subprocess import PIPE
import sharedVariables
from sharedVariables import *

# Network Logs

In [7]:

credentials = service_account.Credentials.from_service_account_file(
    '../infrastructure/credentials.json')

client = bigquery.Client(credentials=credentials)
today = datetime.datetime.today()
table_id = f"compute_googleapis_com_vpc_flows_{today.strftime('%Y%m%d')}"
# table_id = 'compute_googleapis_com_vpc_flows_20210720'
blob_name = "export.log"



extract_conf = ExtractJobConfig()
extract_conf.compression = 'NONE'
extract_conf.destination_format = 'NEWLINE_DELIMITED_JSON'

def getDataset(experiment, outDir = datetime.datetime.today().strftime('%Y%m%d-%H')): 
    ensureDirectory(outDir)
    bucket_name = experiment + "-log-bucket"

    destination_uri = "gs://{}/{}".format(bucket_name, blob_name)
    dataset_ref = bigquery.DatasetReference(project, experiment.replace("-", "_"))
    table_ref = dataset_ref.table(table_id)

    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
        job_config=extract_conf
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print(
        "Exported {}:{}.{} to {}".format(project, experiment, table_id, destination_uri)
    )


    storage_client = storage.Client(credentials=credentials)

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(f"{outDir}/gcp-flow-network.log")

    print(
        "Blob downloaded successfully."
    )

if 'experiment' not in locals():
    for experiment in sharedVariables.experiments: 
        try: 
            getDataset(experiment)
        except Exception as e: 
            print(e)

Exported dspj-315716:experiment-baseline-with-latency-3.compute_googleapis_com_vpc_flows_20210920 to gs://experiment-baseline-with-latency-3-log-bucket/export.log
Blob downloaded successfully.
404 Not found: Table dspj-315716:experiment_syncmesh_with_latency_3.compute_googleapis_com_vpc_flows_20210920 was not found in location US
404 Not found: Table dspj-315716:experiment_advanced_mongo_with_latency_3.compute_googleapis_com_vpc_flows_20210920 was not found in location US
404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dspj-315716/jobs?prettyPrint=false: Not found: Dataset dspj-315716:experiment_syncmesh_with_latency_6
404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dspj-315716/jobs?prettyPrint=false: Not found: Dataset dspj-315716:experiment_baseline_with_latency_6


# PCAPs

In [27]:
def download_pcap(ip, outDir = datetime.datetime.today().strftime('%Y%m%d-%H')):
    ensureDirectory(outDir)
    print("Dowloading File")
    print(os.path.join(Path.cwd(), "..", "cert"))
    result = subprocess.run(f"scp -i ../infrastructure/orchestrator.pem -o StrictHostKeyChecking=no orchestrator@{ip}:/captures.zip ./captures.zip", shell=True, stdout=PIPE, stderr=PIPE, universal_newlines=True)
    if(result.returncode == 0):
        print(result.stdout)
    else:
        raise Exception(result.stderr)
    shutil.unpack_archive("./captures.zip", outDir)

if 'experiment' not in locals():
    download_pcap("35.224.133.98")


# Monitoring

In [97]:
credentials = service_account.Credentials.from_service_account_file(
    '../infrastructure/credentials.json')

# Our project ID

monitoring_client = monitoring_v3.MetricServiceClient(credentials=credentials)
project_name = f"projects/{project_id}"
now = time.time()
seconds = int(now)
nanos = int((now - seconds) * 10 ** 9)

# 3600 = Get the last hour of metrics
timeframe = 3600 * 1
if 'experiment' not in locals():
    print("set timeframe to one hour")
    timeframe = 3600 * 1

interval = monitoring_v3.TimeInterval(
    {
        "end_time": {"seconds": seconds, "nanos": nanos},
        "start_time": {"seconds": (seconds - timeframe), "nanos": nanos},
    }
)

# Add Filter? metric.label.instance_name = "your-instance-id"
results_cpu = monitoring_client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type =  "compute.googleapis.com/instance/cpu/utilization"',
        "interval": interval,
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)
results_io_read = monitoring_client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type = "compute.googleapis.com/instance/disk/read_bytes_count"',
        "interval": interval,
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)
results_io_write = monitoring_client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type = "compute.googleapis.com/instance/disk/write_bytes_count"',
        "interval": interval,
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)
results_iops_read = monitoring_client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type = "compute.googleapis.com/instance/disk/read_ops_count"',
        "interval": interval,
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)
results_iops_write = monitoring_client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type = "compute.googleapis.com/instance/disk/write_ops_count"',
        "interval": interval,
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)

mutli_results_memory = {}
mem_states = ["buffered", "cached", "free", "slab", "used"]
for state in mem_states:
    mutli_results_memory[state] = monitoring_client.list_time_series(
        request={
            "name": project_name,
            "filter": f'metric.type = "agent.googleapis.com/memory/bytes_used" AND metric.labels.state = "{state}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
        }
    )

mutli_results_memory_percentage = {}
for state in mem_states:
    mutli_results_memory_percentage[state] = monitoring_client.list_time_series(
        request={
            "name": project_name,
            "filter": f'metric.type = "agent.googleapis.com/memory/percent_used" AND metric.labels.state = "{state}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
        }
    )



# Prepare Data so that as the monitoring time_series_list is not aligned
time_series_list_list_labels = ["cpu_util_", "io_read_", "io_write_", "iops_read_", "iops_write_"]
time_series_list_list = [results_cpu, results_io_read, results_io_write, results_iops_read, results_iops_write]
# Add Memory
for state in mem_states:
    time_series_list_list.append(mutli_results_memory[state])
    time_series_list_list_labels.append(f"mem_{state}_")

for state in mem_states:
    time_series_list_list.append(mutli_results_memory_percentage[state])
    time_series_list_list_labels.append(f"mem_perc_{state}_")

print(time_series_list_list_labels)

instance_logs = {}

for index, time_series_list in enumerate(time_series_list_list):
    for time_series in time_series_list:
        if time_series.metric.labels["instance_name"]:
            label = time_series.metric.labels["instance_name"]
        else:
            # Find out label by comapring the instance ID
            for compare_item in time_series_list_list[0]:
                if compare_item.resource.labels["instance_id"] == time_series.resource.labels["instance_id"]:
                    label = compare_item.metric.labels["instance_name"]
        if label not in instance_logs:
            instance_logs[label] = np.empty(len(time_series_list_list), dtype=object)
        instance_logs[label][index] = time_series

def loadMonitoringData(experiment, outDir = datetime.datetime.today().strftime('%Y%m%d-%H')): 
    ensureDirectory(outDir)
    df = pd.DataFrame()
    set_timestamp_column = True
    first_len = 0
    # Filter instances for our experiment
    filteredInstances = {}
    for key, instance in instance_logs.items():
        if key.startswith(experiment):
            filteredInstances[key] = instance

    # Find out which point array is smallest
    first_len = min([len(item.points) for instances in filteredInstances.values() for item in instances])
    for instances in filteredInstances.values():
        print(instances[0].metric.labels["instance_name"], len(instances[0].points))



    print("Overall min len:", first_len)

    for key, instances in filteredInstances.items():
        for index in range(len(instances)):
            label = key
            # if "orchestrator" in label:
            #     # Skip Orchesrtator
            #     continue
            if set_timestamp_column:
                # first_len = min([len(item.points) - 1 for item in instances])
                # print(first_len)
                print("SET min len:", first_len)

                df['timestamp'] = pd.to_datetime([p.interval.start_time.ToDatetime() for p in instances[0].points[:first_len]])
                set_timestamp_column = False
            # print(ts.points[0])
            # When deploying the vm they might take different amount of time leading to some values beeing available a minute early this leading to different length
            # We can trim off the last values as they are orderer from most recent to last
            df[time_series_list_list_labels[index] + label] = [p.value.double_value for p in instances[index].points[:first_len]]

            # df['cpu_util_' + label] = [p.value.double_value for p in ts_cpu.points[:first_len]]
            # df['io_read_' + label] = [p.value.int64_value for p in ts_io_read.points[:first_len]]
            # df['io_write_' + label] = [p.value.int64_value for p in ts_io_write.points[:first_len]]
            # df['iops_read_' + label] = [p.value.int64_value for p in ts_iops_read.points[:first_len]]
            # df['iops_write_' + label] = [p.value.int64_value for p in ts_iops_write.points[:first_len]]

    df.set_index('timestamp', inplace=True)
    df.index = pd.to_datetime(df.index)
    columns = df.columns
    df.to_csv(f"{outDir}/monitoring.csv")
    print(f"Gathered all Monitoring data for {experiment}")
    return df


# Make sure your experiment name is included in the experiments list, otherwise only errors will occur.
if 'experiment' not in locals():
    for experiment in sharedVariables.experiments: 
        try: 
            loadMonitoringData(experiment)
        except Exception as e: 
            print(e)

['cpu_util_', 'io_read_', 'io_write_', 'iops_read_', 'iops_write_', 'mem_buffered_', 'mem_cached_', 'mem_free_', 'mem_slab_', 'mem_used_', 'mem_perc_buffered_', 'mem_perc_cached_', 'mem_perc_free_', 'mem_perc_slab_', 'mem_perc_used_']
experiment-advanced-mongo-with-latency-6-node-instance-2 51
experiment-advanced-mongo-with-latency-6-node-instance-6 52
experiment-advanced-mongo-with-latency-6-node-instance-4 52
experiment-advanced-mongo-with-latency-6-node-instance-3 52
experiment-advanced-mongo-with-latency-6-node-instance-1 51
experiment-advanced-mongo-with-latency-6-node-instance-5 52
experiment-advanced-mongo-with-latency-6-client-instance 51
experiment-advanced-mongo-with-latency-6-central-server 51
experiment-advanced-mongo-with-latency-6-test-orchestrator 50
experiment-advanced-mongo-with-latency-6-central-config-server 52
Overall min len: 50
SET min len: 50
Gathered all Monitoring data for experiment-advanced-mongo-with-latency-6


C:\Users\danie\AppData\Local\Temp/ipykernel_32556/1718896806.py:160: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[time_series_list_list_labels[index] + label] = [p.value.double_value for p in instances[index].points[:first_len]]


,cpu_util_experiment-advanced-mongo-with-latency-6-node-instance-2,io_read_experiment-advanced-mongo-with-latency-6-node-instance-2,io_write_experiment-advanced-mongo-with-latency-6-node-instance-2,iops_read_experiment-advanced-mongo-with-latency-6-node-instance-2,iops_write_experiment-advanced-mongo-with-latency-6-node-instance-2,mem_buffered_experiment-advanced-mongo-with-latency-6-node-instance-2,mem_cached_experiment-advanced-mongo-with-latency-6-node-instance-2,mem_free_experiment-advanced-mongo-with-latency-6-node-instance-2,mem_slab_experiment-advanced-mongo-with-latency-6-node-instance-2,mem_used_experiment-advanced-mongo-with-latency-6-node-instance-2,...,mem_buffered_experiment-advanced-mongo-with-latency-6-central-config-server,mem_cached_experiment-advanced-mongo-with-latency-6-central-config-server,mem_free_experiment-advanced-mongo-with-latency-6-central-config-server,mem_slab_experiment-advanced-mongo-with-latency-6-central-config-server,mem_used_experiment-advanced-mongo-with-latency-6-central-config-server,mem_perc_buffered_experiment-advanced-mongo-with-latency-6-central-config-server,mem_perc_cached_experiment-advanced-mongo-with-latency-6-central-config-server,mem_perc_free_experiment-advanced-mongo-with-latency-6-central-config-server,mem_perc_slab_experiment-advanced-mongo-with-latency-6-central-config-server,mem_perc_used_experiment-advanced-mongo-with-latency-6-central-config-server
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-09-29 15:42:00,0.083121,0.0,0.0,0.0,0.0,50823168.0,1.605366e+09,1.808052e+09,120836096.0,270245888.0,...,54259712.0,1.587548e+09,1.858732e+09,119803904.0,234979328.0,1.407397,41.178083,48.212094,3.107493,6.094932
2021-09-29 15:41:00,0.021420,0.0,0.0,0.0,0.0,50642944.0,1.605317e+09,1.807503e+09,120840192.0,271020032.0,...,54067200.0,1.587479e+09,1.859334e+09,119771136.0,234672128.0,1.402404,41.176277,48.227712,3.106643,6.086964
2021-09-29 15:40:00,0.017396,0.0,0.0,0.0,0.0,50511872.0,1.605267e+09,1.807303e+09,120832000.0,271409152.0,...,53895168.0,1.587323e+09,1.859748e+09,119758848.0,234598400.0,1.397942,41.172240,48.238443,3.106325,6.085052
2021-09-29 15:39:00,0.018158,0.0,0.0,0.0,0.0,50397184.0,1.605218e+09,1.807790e+09,120840192.0,271077376.0,...,53694464.0,1.587171e+09,1.860194e+09,119754752.0,234508288.0,1.392736,41.168309,48.250023,3.106218,6.082714
2021-09-29 15:38:00,0.036292,0.0,0.0,0.0,0.0,50253824.0,1.605169e+09,1.812070e+09,120803328.0,267026432.0,...,53510144.0,1.586872e+09,1.863541e+09,119689216.0,231710720.0,1.387955,41.160553,48.336823,3.104518,6.010150
2021-09-29 15:37:00,0.017840,0.0,0.0,0.0,0.0,50139136.0,1.605120e+09,1.812062e+09,120811520.0,267190272.0,...,53338112.0,1.586749e+09,1.863664e+09,119664640.0,231907328.0,1.383493,41.157366,48.340011,3.103881,6.015250
2021-09-29 15:36:00,0.018969,0.0,0.0,0.0,0.0,50020352.0,1.605075e+09,1.811771e+09,120786944.0,267669504.0,...,53186560.0,1.586618e+09,1.864094e+09,119672832.0,231751680.0,1.379562,41.153966,48.351166,3.104093,6.011213
2021-09-29 15:35:00,0.024715,0.0,0.0,0.0,0.0,49889280.0,1.605030e+09,1.811313e+09,120791040.0,268300288.0,...,52953088.0,1.586471e+09,1.864491e+09,119652352.0,231755776.0,1.373506,41.150141,48.361472,3.103562,6.011319
2021-09-29 15:34:00,0.030780,0.0,0.0,0.0,0.0,49790976.0,1.604526e+09,1.805222e+09,120958976.0,274825216.0,...,52764672.0,1.586336e+09,1.864618e+09,119726080.0,231878656.0,1.368619,41.146635,48.364765,3.105475,6.014506


# Get TimeStamps

In [23]:
# # Workaround for now
# # df_sync3 = pd.read_csv(f'{outdir}/experiment-syncmesh-with-latency-3.csv')
# # df_base3 = pd.read_csv(f'{outdir}/experiment-baseline-with-latency-3.csv')
# df_sync3 = loadData(f'{outdir}/experiment-syncmesh-with-latency-3.log')
# df_base3 = loadData(f'{outdir}/experiment-baseline-with-latency-3.log')
# # df_base3.set_index('timestamp', inplace=True)
# # df_sync3.set_index('timestamp', inplace=True)

# # df_sync6 = loadData(f'{outdir}/experiment-syncmesh-with-latency-6.log')
# # df_base6 = loadData(f'{outdir}/experiment-baseline-with-latency-6.log')

# def filterDataForSeperator(df): 
#     df = df[df["jsonPayload.connection.src_ip"].isin([ip_seperator, ip_orchestrator])]
#     df = df[df["jsonPayload.connection.dest_ip"].isin([ip_seperator, ip_orchestrator])]
#     df = df[df["jsonPayload.connection.dest_port"] == 443]
#     return df

# seperator_base3 = filterDataForSeperator(df_base3)
# seperator_sync3 = filterDataForSeperator(df_sync3)

# # df = df[df["jsonPayload.connection.dest_ip"]]
# print(seperator_sync3.index)
# print(seperator_base3.index)
# # df_base3.info()

In [30]:
# Script for automatic data retrieval via papermill
if 'experiment' in locals():
    todaystring = datetime.datetime.today().strftime('%Y%m%d-%H')
    outdir = f"data/{todaystring}-{experiment}"

    f = open(os.path.join(Path.cwd(), "..", "infrastructure", "orchestrator.txt"), "r")
    ip = f.read()
    download_pcap(ip, outDir=outdir)
    loadMonitoringData(experiment, outDir=outdir)
    # getDataset(experiment, outDir=outdir)

    

Dowloading File
c:\Develop\GitHub\DSPJ2021\syncmesh\evaluation\..\cert

experiment-advanced-mongo-with-latency-3-node-instance-2: 165
164
experiment-advanced-mongo-with-latency-3-node-instance-2: 36


ValueError: Length of values (36) does not match length of index (164)